## Setting up

In [1]:
# %%capture
# %pip install -U bitsandbytes
# %pip install -U transformers
# %pip install -U accelerate
# %pip install -U peft
# %pip install -U trl

In [2]:
# import wandb

# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()

# wb_token = user_secrets.get_secret("wandb")

# wandb.login(key=wb_token)
# run = wandb.init(
#     project='Fine-tune llama-3.1-8b-it on Sentiment Analysis Dataset',
#     job_type="training",
#     anonymous="allow"
# )

In [3]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

/root/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/root/miniconda3/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Loading and processing the dataset

In [4]:
%pwd

'/root/autodl-tmp'

In [5]:
df = pd.read_json(r'/root/reinforcement_commit/datasets/patch_db.json', encoding='utf_8_sig')
df.dropna(inplace=True)
label2id={'non-security':0,'security':1}
df = df.replace({"category": label2id})
df.rename(columns={'category':'label','diff_code':'diff','commit_message':'message'},inplace=True)
df
label2id={0:'negative',1:'positive'}
df = df.replace({"label": label2id})
df =  df[df['diff'].str.len()<1024]
df 

/tmp/ipykernel_2854/4172699659.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"category": label2id})


,CVE_ID,CWE_ID,label,commit_id,message,diff,owner,repo,source
2,NA,NA,negative,f181dd278274f50e689ebd13237010a90b430164,commit f181dd278274f50e689ebd13237010a90b43016...,diff --git a/include/paths.h b/include/paths.h...,openbsd,src,wild
3,NA,NA,negative,0abdc3723b5d33dde698ab941325edec2819c128,commit 0abdc3723b5d33dde698ab941325edec2819c12...,diff --git a/gnu/usr.bin/binutils/ld/lexsup.c ...,openbsd,src,wild
4,NA,NA,negative,d7930d7f820e5dd6b07b823f155aeb943b525e16,commit d7930d7f820e5dd6b07b823f155aeb943b525e1...,diff --git a/src/expat_erl.c b/src/expat_erl.c...,esl,MongooseIM,wild
5,NA,NA,negative,570597952c2f88cfaaee92a7c0a852ef9924ca5c,commit 570597952c2f88cfaaee92a7c0a852ef9924ca5...,diff --git a/proto.c b/proto.c\nindex 6a788cc5...,wireshark,wireshark,wild
7,NA,NA,negative,88674088d10ca2538b2efd2559f6620ade8ec373,commit 88674088d10ca2538b2efd2559f6620ade8ec37...,diff --git a/arch/x86/video/fbdev.c b/arch/x86...,torvalds,linux,wild
...,...,...,...,...,...,...,...,...,...
35797,CVE-2013-1959,264,positive,e3211c120a85b792978bcb4be7b2886df18d27f0,From e3211c120a85b792978bcb4be7b2886df18d27f0 ...,diff --git a/kernel/user_namespace.c b/kernel/...,torvalds,linux,cve
35798,CVE-2015-5364,399,positive,beb39db59d14990e401e235faf66a6b9b31240b0,From beb39db59d14990e401e235faf66a6b9b31240b0 ...,diff --git a/net/ipv4/udp.c b/net/ipv4/udp.c\n...,torvalds,linux,cve
35805,CVE-2017-7184,264,positive,677e806da4d916052585301785d847c3b3e6186a,From 677e806da4d916052585301785d847c3b3e6186a ...,diff --git a/net/xfrm/xfrm_user.c b/net/xfrm/x...,torvalds,linux,cve
35806,CVE-2013-3672,20,positive,8d3c99e825317b7efda5fd12e69896b47c700303,From 8d3c99e825317b7efda5fd12e69896b47c700303 ...,diff --git a/libavcodec/mmvideo.c b/libavcodec...,NA,NA,cve


In [6]:
# Shuffle the DataFrame and select only 3000 rows
df = df.sample(frac=1, random_state=85).reset_index(drop=True).head(3000)

# Split the DataFrame
train_size = 0.8
eval_size = 0.1

# Calculate sizes
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))

# Split the data
X_train = df[:train_end]
X_eval = df[train_end:eval_end]
X_test = df[eval_end:]

# Define the prompt generation functions
def generate_prompt(data_point):
    return f"""
            Classify the text into negative, positive, and return the answer as the corresponding security patch identification label.
text: {data_point["diff"]}
label: {data_point["label"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
            Classify the text into negative, positive, and return the answer as the corresponding security patch identification label.
text: {data_point["diff"]}
label: """.strip()

# Generate prompts for training and evaluation data
X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)

# Generate test prompts and extract true labels
y_true = X_test.loc[:,'label']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])



/tmp/ipykernel_2854/176985973.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
/tmp/ipykernel_2854/176985973.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)


In [7]:
X_train.label.value_counts()

label
negative    1481
positive     919
Name: count, dtype: int64

In [8]:
y_true.value_counts()

label
negative    186
positive    114
Name: count, dtype: int64

In [9]:
# Convert to datasets
train_data = Dataset.from_pandas(X_train[["text"]])
eval_data = Dataset.from_pandas(X_eval[["text"]])

In [10]:
train_data['text'][3]

"Classify the text into negative, positive, and return the answer as the corresponding security patch identification label.\ntext: diff --git a/src/common/tortls.c b/src/common/tortls.c\nindex 0571b346d..e052c85eb 100644\n--- a/src/common/tortls.c\n+++ b/src/common/tortls.c\n@@ -1318,7 +1318,8 @@ tor_tls_got_client_hello(tor_tls_t *tls)\n     }\n \n     tls->got_renegotiate = 1;\n-  } else if (tls->server_handshake_count > 2) {\n+  } else if (tls->server_handshake_count > 2 &&\n+             tls->excess_renegotiations_callback) {\n     /* We got more than one renegotiation requests. The Tor protocol\n        needs just one renegotiation; more than that probably means\n        They are trying to DoS us and we have to stop them. We can't\n\n\nlabel: negative"

## Loading the model and tokenizer

In [11]:
base_model_name = "/root/autodl-tmp/models/Llama3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id

## Model evalution before fine-tuning

In [13]:
def predict(test, model, tokenizer):
    y_pred = []
    categories = ["negative", "positive"]

    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=2,
                        temperature=0.1)

        result = pipe(prompt)
        answer = result[0]['generated_text'].split("label:")[-1].strip()

        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")

    return y_pred

In [14]:
y_pred = predict(X_test, model, tokenizer)

100%|██████████| 300/300 [00:41<00:00,  7.19it/s]


In [15]:
def evaluate(y_true, y_pred):
    labels = ["negative", "positive"]
    mapping = {label: idx for idx, label in enumerate(labels)}

    def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found, but should not occur with correct data

    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true_mapped)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {label_accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true_mapped, y_pred=y_pred_mapped, target_names=labels, digits=4, labels=list(range(len(labels))))
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true_mapped, y_pred=y_pred_mapped, labels=list(range(len(labels))))
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [16]:
evaluate(y_true, y_pred)

Accuracy: 0.107
Accuracy for label negative: 0.070
Accuracy for label positive: 0.167

Classification Report:
              precision    recall  f1-score   support

    negative     0.5652    0.0699    0.1244       186
    positive     0.3519    0.1667    0.2262       114

   micro avg     0.4156    0.1067    0.1698       300
   macro avg     0.4585    0.1183    0.1753       300
weighted avg     0.4841    0.1067    0.1631       300


Confusion Matrix:
[[13 35]
 [10 19]]


## Extracting the linear modules names

In [17]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [18]:
modules = find_all_linear_names(model)
modules

['o_proj', 'gate_proj', 'down_proj', 'k_proj', 'v_proj', 'q_proj', 'up_proj']

## Setting up the model

In [19]:
output_dir="llama-3.1-fine-tuned-model"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=1,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="none",                  # report metrics to w&b
    eval_strategy="steps",              # save checkpoint every epoch
    eval_steps = 0.2
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    processing_class=tokenizer
)

Adding EOS to train dataset:   0%|          | 0/2400 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2400 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2400 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

## Model Training

In [20]:
# Train model
trainer.train()

Step,Training Loss,Validation Loss
60,1.291800,1.201978
120,1.208700,1.189090
180,1.233000,1.176059
240,1.260000,1.170624
300,1.172900,1.169208


TrainOutput(global_step=300, training_loss=1.2282722266515096, metrics={'train_runtime': 1458.4078, 'train_samples_per_second': 1.646, 'train_steps_per_second': 0.206, 'total_flos': 2.627464390319309e+16, 'train_loss': 1.2282722266515096})

In [21]:
# wandb.finish()
model.config.use_cache = True

## Saving the model and tokenizer

In [22]:
# Save trained model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('llama-3.1-fine-tuned-model/tokenizer_config.json',
 'llama-3.1-fine-tuned-model/special_tokens_map.json',
 'llama-3.1-fine-tuned-model/chat_template.jinja',
 'llama-3.1-fine-tuned-model/tokenizer.json')

## Testing model after fine-tuning

In [23]:
y_pred = predict(X_test, model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████| 300/300 [01:13<00:00,  4.08it/s]

Accuracy: 0.663
Accuracy for label negative: 0.989
Accuracy for label positive: 0.132

Classification Report:
              precision    recall  f1-score   support

    negative     0.6502    0.9892    0.7846       186
    positive     0.8824    0.1316    0.2290       114

    accuracy                         0.6633       300
   macro avg     0.7663    0.5604    0.5068       300
weighted avg     0.7384    0.6633    0.5735       300


Confusion Matrix:
[[184   2]
 [ 99  15]]
